# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from scipy.signal import butter, filtfilt
import random

In [36]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/AveragePooled/split_features_1s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [37]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [38]:
train_data

{'melspectrogram': array([[-0.01720456, -0.9333485 , -0.9391832 , ..., -0.82813597,
         -0.8620985 , -0.9794059 ],
        [-0.01720306, -0.9333132 , -0.93914014, ..., -0.8327555 ,
         -0.86637986, -0.97935015],
        [-0.01720168, -0.9332716 , -0.93908674, ..., -0.847576  ,
         -0.8784226 , -0.9896085 ],
        ...,
        [-0.01720948, -0.55143994, -0.52205884, ..., -0.87508094,
         -0.90587527, -0.9997576 ],
        [-0.0171332 , -0.55125844, -0.51631904, ..., -0.8850505 ,
         -0.9144034 , -0.99985427],
        [-0.01714398, -0.5486961 , -0.5132266 , ..., -0.88104194,
         -0.911118  , -1.        ]], dtype=float32),
 'mfcc': array([[-1.        , -0.02898505, -0.10768069, ...,  0.01827751,
          0.02722204,  0.04308926],
        [-1.        , -0.02902972, -0.1065027 , ...,  0.01701666,
          0.02487057,  0.04410085],
        [-1.        , -0.0125336 , -0.09841443, ...,  0.02036034,
          0.02327715,  0.04241924],
        ...,
        [-0.9

In [39]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [40]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [41]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [42]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [43]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [44]:
display(train_y.shape)
display(train_y[:15])

(42578,)

array([ 7, 16, 19,  0,  1,  7,  6,  3,  1,  1, 19,  7,  9,  7, 12],
      dtype=int32)

In [45]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(42578, 128)

array([-0.02185728, -0.3939447 , -0.41050395, -0.50029534, -0.56085765,
       -0.6310321 , -0.6548134 , -0.6388136 , -0.63982546, -0.6337072 ,
       -0.64285135, -0.65793073, -0.6480776 , -0.6594148 , -0.6366135 ,
       -0.654948  , -0.6432404 , -0.6395858 , -0.6798748 , -0.6723758 ,
       -0.66720843, -0.69844174, -0.7145976 , -0.7270461 , -0.7289734 ,
       -0.75110966, -0.7298497 , -0.696753  , -0.68113   , -0.683587  ,
       -0.6695822 , -0.66871876, -0.66008353, -0.6631871 , -0.65656364,
       -0.6460304 , -0.6575156 , -0.6447011 , -0.6472018 , -0.6569579 ,
       -0.66957086, -0.67637694, -0.67121893, -0.6779927 , -0.68991876,
       -0.6964    , -0.7139648 , -0.7335337 , -0.7551437 , -0.7620292 ,
       -0.76579344, -0.7464363 , -0.71748465, -0.6991841 , -0.6900108 ,
       -0.6999048 , -0.70721406, -0.7059485 , -0.6748798 , -0.66050416,
       -0.6367882 , -0.62208366, -0.6234393 , -0.6094999 , -0.595338  ,
       -0.602499  , -0.62069196, -0.66656387, -0.7080173 , -0.73

'mfcc'

(42578, 20)

...

(42578, 84)

array([-12.80637 , -10.178921, -15.586308, -16.283943, -17.871552,
       -15.207264, -15.745362, -16.976467, -19.377678, -20.634443,
       -19.823278, -16.28474 , -11.230347, -13.031438, -22.283443,
       -23.941479, -24.802055, -24.640802, -26.405006, -22.172436,
       -23.847929, -29.4433  , -32.76458 , -32.2666  , -29.44668 ,
       -31.624424, -37.031097, -38.127777, -39.327816, -38.604076,
       -38.72474 , -38.8844  , -37.452694, -36.85629 , -38.343513,
       -37.406395, -37.946125, -37.184864, -38.44911 , -39.839275,
       -38.192352, -39.987495, -38.41094 , -37.54097 , -38.557415,
       -38.12515 , -37.687935, -40.436245, -40.11137 , -41.43069 ,
       -43.12546 , -45.00774 , -46.521675, -45.079754, -42.41699 ,
       -41.021828, -39.632954, -39.205273, -38.933006, -37.991035,
       -38.70373 , -40.62161 , -40.877434, -43.137257, -46.641926,
       -46.76184 , -43.449017, -42.309822, -41.863094, -38.54685 ,
       -35.893127, -34.787994, -36.57433 , -41.203064, -46.978

'id'

(42578,)

'XC184429.mp3'

In [46]:
display(val_y.shape)
display(val_y[:15])

(13083,)

array([ 3,  5, 11, 11, 18, 12, 19, 18,  1,  2,  1,  4, 17,  0, 11],
      dtype=int32)

In [47]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(13083, 128)

array([-0.01719964, -0.9332614 , -0.9346287 , -0.92839473, -0.9182133 ,
       -0.91571504, -0.90023047, -0.9096828 , -0.9013315 , -0.8832822 ,
       -0.8897478 , -0.8778112 , -0.8611843 , -0.87501085, -0.86042863,
       -0.87427056, -0.8929696 , -0.8929555 , -0.89963925, -0.90093535,
       -0.90265375, -0.90664506, -0.9189995 , -0.91942257, -0.9140625 ,
       -0.9264467 , -0.93026924, -0.9284385 , -0.934406  , -0.92323446,
       -0.93527853, -0.9328583 , -0.9292407 , -0.9284236 , -0.9269353 ,
       -0.9171119 , -0.9047776 , -0.8865346 , -0.90121865, -0.90096414,
       -0.9121269 , -0.91248345, -0.893796  , -0.90334725, -0.914868  ,
       -0.9018331 , -0.8862433 , -0.8832799 , -0.8790057 , -0.87130845,
       -0.86195666, -0.8688272 , -0.8709505 , -0.85542107, -0.8491089 ,
       -0.8495483 , -0.8412429 , -0.84774375, -0.84960586, -0.85435015,
       -0.85175586, -0.85534483, -0.8512846 , -0.85755765, -0.8516211 ,
       -0.8539241 , -0.8570129 , -0.83969903, -0.8464547 , -0.84

'mfcc'

(13083, 20)

...

(13083, 84)

array([-35.882477, -38.292088, -37.4063  , -40.330284, -42.694447,
       -42.67396 , -44.58036 , -46.30381 , -47.487316, -47.476955,
       -48.25535 , -50.481647, -55.270306, -54.9184  , -54.514065,
       -53.466293, -53.302032, -54.178078, -56.668327, -56.69416 ,
       -56.8258  , -58.064236, -57.938488, -56.90294 , -56.90026 ,
       -56.48227 , -58.99698 , -58.85235 , -58.181664, -55.98642 ,
       -57.59114 , -58.67091 , -59.410954, -58.078316, -58.395626,
       -56.53549 , -56.802666, -57.817825, -57.55971 , -57.068478,
       -55.886612, -55.62043 , -56.85259 , -55.615112, -57.423187,
       -58.65168 , -58.25182 , -58.85849 , -58.846207, -59.66834 ,
       -60.100693, -60.817085, -61.510292, -61.84868 , -62.075718,
       -62.074112, -62.60141 , -62.69142 , -60.156475, -58.66417 ,
       -58.836452, -58.94244 , -59.174664, -58.008244, -57.496716,
       -56.099037, -56.071007, -54.518993, -54.06901 , -54.937103,
       -54.929108, -54.014652, -54.42614 , -54.037262, -54.998

'id'

(13083,)

'XC511677.mp3'

## **Random Forest Model**

Saving evaluation results

In [48]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    # val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = None # roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted') Look at documentation
    # print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [49]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [50]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(42578, 244)

In [51]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(13083, 244)

Fit the model with training data

In [52]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [53]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 0.7517027572924985
Validation accuracy: 0.2198272567453948


In [54]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.10      0.19      0.13       793
           1       0.47      0.49      0.48      1294
           2       0.23      0.36      0.28       617
           3       0.15      0.32      0.21       984
           4       0.04      0.03      0.04       229
           5       0.00      0.00      0.00       303
           6       0.11      0.01      0.02       275
           7       0.31      0.42      0.36       698
           8       0.09      0.14      0.11       176
           9       0.23      0.21      0.22       227
          10       0.20      0.12      0.15       455
          11       0.24      0.03      0.05      1660
          12       0.35      0.32      0.33      1261
          13       0.49      0.19      0.28       502
          14       0.01      0.03      0.01       156
          15       0.16      0.23      0.19       798
          16       0.46      0.30      0.36   

### **With MFCCs only**

In [55]:
training_features = train_features['mfcc']

training_features.shape

(42578, 20)

In [56]:
validation_features = val_features['mfcc']

validation_features.shape

(13083, 20)

In [57]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [58]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 0.6789186904034947
Validation accuracy: 0.27272032408469005


In [60]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.17      0.44      0.24       793
           1       0.53      0.44      0.48      1294
           2       0.27      0.35      0.30       617
           3       0.26      0.53      0.35       984
           4       0.04      0.04      0.04       229
           5       0.04      0.03      0.03       303
           6       0.16      0.03      0.04       275
           7       0.30      0.42      0.35       698
           8       0.16      0.22      0.18       176
           9       0.21      0.32      0.26       227
          10       0.29      0.31      0.30       455
          11       0.22      0.02      0.04      1660
          12       0.30      0.32      0.31      1261
          13       0.38      0.22      0.28       502
          14       0.06      0.18      0.09       156
          15       0.26      0.25      0.26       798
          16       0.51      0.34      0.41   

### **With Mel-Spectrogram Only**

In [61]:
training_features = train_features['melspectrogram']

training_features.shape

(42578, 128)

In [62]:
validation_features = val_features['melspectrogram']

validation_features.shape

(13083, 128)

In [63]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [64]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 0.9666494433745126
Validation accuracy: 0.27050370710081784


In [65]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.21      0.42      0.28       793
           1       0.40      0.51      0.45      1294
           2       0.35      0.37      0.36       617
           3       0.28      0.47      0.35       984
           4       0.02      0.03      0.02       229
           5       0.06      0.02      0.03       303
           6       0.24      0.11      0.15       275
           7       0.47      0.35      0.40       698
           8       0.12      0.57      0.19       176
           9       0.14      0.31      0.19       227
          10       0.16      0.12      0.13       455
          11       0.17      0.02      0.03      1660
          12       0.39      0.32      0.35      1261
          13       0.43      0.38      0.41       502
          14       0.09      0.12      0.10       156
          15       0.16      0.21      0.18       798
          16       0.30      0.12      0.17   

## **With Chroma Only**

In [66]:
training_features = train_features['chroma']

training_features.shape

(42578, 12)

In [67]:
validation_features = val_features['chroma']

validation_features.shape

(13083, 12)

In [68]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [69]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 0.3582366480341961
Validation accuracy: 0.18107467706183597


In [70]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.08      0.64      0.14       793
           1       0.41      0.33      0.36      1294
           2       0.11      0.41      0.17       617
           3       0.60      0.32      0.42       984
           4       0.06      0.00      0.01       229
           5       0.42      0.11      0.17       303
           6       0.00      0.00      0.00       275
           7       0.23      0.17      0.20       698
           8       0.00      0.00      0.00       176
           9       0.31      0.13      0.18       227
          10       0.00      0.00      0.00       455
          11       0.00      0.00      0.00      1660
          12       0.65      0.05      0.09      1261
          13       0.24      0.01      0.03       502
          14       0.06      0.01      0.01       156
          15       0.28      0.49      0.36       798
          16       0.41      0.20      0.26   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **With CQT Only**

In [71]:
training_features = train_features['cqt']

training_features.shape

(42578, 84)

In [72]:
validation_features = val_features['cqt']

validation_features.shape

(13083, 84)

In [73]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [74]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 0.9630090657146884
Validation accuracy: 0.19842543759076664


In [75]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.10      0.14      0.12       793
           1       0.34      0.54      0.42      1294
           2       0.04      0.06      0.05       617
           3       0.28      0.30      0.29       984
           4       0.08      0.07      0.07       229
           5       0.00      0.00      0.00       303
           6       0.08      0.07      0.07       275
           7       0.36      0.37      0.36       698
           8       0.10      0.48      0.17       176
           9       0.17      0.18      0.17       227
          10       0.17      0.08      0.10       455
          11       0.22      0.01      0.02      1660
          12       0.33      0.31      0.32      1261
          13       0.22      0.19      0.21       502
          14       0.03      0.10      0.04       156
          15       0.11      0.13      0.12       798
          16       0.38      0.12      0.19   

## **With MFCCs and Mel-Spectrogram**

In [76]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(42578, 148)

In [77]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(13083, 148)

In [78]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [79]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 0.8542439757621306
Validation accuracy: 0.28166322708858826


In [80]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.20      0.38      0.26       793
           1       0.51      0.56      0.54      1294
           2       0.30      0.35      0.33       617
           3       0.29      0.55      0.38       984
           4       0.03      0.02      0.02       229
           5       0.05      0.04      0.05       303
           6       0.24      0.03      0.05       275
           7       0.38      0.35      0.37       698
           8       0.11      0.47      0.18       176
           9       0.15      0.32      0.20       227
          10       0.24      0.21      0.23       455
          11       0.27      0.03      0.05      1660
          12       0.41      0.35      0.38      1261
          13       0.24      0.19      0.21       502
          14       0.03      0.06      0.04       156
          15       0.21      0.29      0.24       798
          16       0.37      0.24      0.29   

## **With MFCCs and Chroma**

In [81]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(42578, 32)

In [82]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(13083, 32)

In [83]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [84]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 0.6089294941049368
Validation accuracy: 0.3096384621264236


In [85]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.14      0.57      0.23       793
           1       0.52      0.44      0.48      1294
           2       0.29      0.40      0.33       617
           3       0.49      0.48      0.48       984
           4       0.10      0.04      0.06       229
           5       0.09      0.06      0.08       303
           6       0.04      0.00      0.01       275
           7       0.26      0.45      0.33       698
           8       0.07      0.06      0.07       176
           9       0.66      0.35      0.46       227
          10       0.29      0.25      0.27       455
          11       0.51      0.02      0.03      1660
          12       0.24      0.24      0.24      1261
          13       0.51      0.34      0.41       502
          14       0.06      0.13      0.08       156
          15       0.46      0.41      0.44       798
          16       0.65      0.79      0.71   

## **With MFCCs and CQT**

In [86]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(42578, 104)

In [87]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(13083, 104)

In [88]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [89]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 0.8617830804640895
Validation accuracy: 0.21187800963081863


In [90]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.12      0.20      0.15       793
           1       0.36      0.51      0.42      1294
           2       0.15      0.22      0.17       617
           3       0.19      0.29      0.23       984
           4       0.09      0.11      0.10       229
           5       0.00      0.00      0.00       303
           6       0.10      0.04      0.05       275
           7       0.37      0.42      0.39       698
           8       0.18      0.45      0.25       176
           9       0.20      0.22      0.21       227
          10       0.22      0.10      0.14       455
          11       0.23      0.02      0.04      1660
          12       0.34      0.33      0.34      1261
          13       0.43      0.17      0.24       502
          14       0.02      0.08      0.03       156
          15       0.14      0.20      0.17       798
          16       0.38      0.16      0.22   

## **With Chroma and CQT**

In [91]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(42578, 96)

In [92]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(13083, 96)

In [93]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [94]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 0.8885574709944103
Validation accuracy: 0.20943208744171826


In [95]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.12      0.18      0.14       793
           1       0.35      0.52      0.42      1294
           2       0.11      0.17      0.13       617
           3       0.22      0.29      0.25       984
           4       0.11      0.11      0.11       229
           5       0.00      0.00      0.00       303
           6       0.08      0.04      0.06       275
           7       0.38      0.42      0.40       698
           8       0.15      0.44      0.22       176
           9       0.22      0.23      0.22       227
          10       0.23      0.11      0.15       455
          11       0.21      0.02      0.03      1660
          12       0.34      0.33      0.33      1261
          13       0.40      0.16      0.23       502
          14       0.01      0.06      0.02       156
          15       0.13      0.19      0.15       798
          16       0.38      0.14      0.21   

## **With Mel-Spectrogram and Chroma**

In [96]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(42578, 140)

In [97]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(13083, 140)

In [98]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [99]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 0.787448917281225
Validation accuracy: 0.3046701826798135


In [100]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.18      0.42      0.25       793
           1       0.55      0.52      0.53      1294
           2       0.25      0.30      0.28       617
           3       0.28      0.55      0.37       984
           4       0.07      0.03      0.04       229
           5       0.08      0.07      0.07       303
           6       0.18      0.01      0.01       275
           7       0.32      0.36      0.34       698
           8       0.10      0.26      0.14       176
           9       0.31      0.33      0.32       227
          10       0.35      0.37      0.36       455
          11       0.25      0.02      0.04      1660
          12       0.35      0.32      0.33      1261
          13       0.28      0.19      0.23       502
          14       0.04      0.12      0.06       156
          15       0.34      0.38      0.36       798
          16       0.55      0.61      0.58   

## **With Mel-Spectrogram and CQT**

In [101]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(42578, 212)

In [102]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(13083, 212)

In [103]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [104]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 0.7617548968951101
Validation accuracy: 0.22082091263471681


In [105]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.10      0.20      0.14       793
           1       0.46      0.50      0.48      1294
           2       0.23      0.36      0.28       617
           3       0.16      0.32      0.21       984
           4       0.04      0.04      0.04       229
           5       0.00      0.00      0.00       303
           6       0.10      0.01      0.02       275
           7       0.31      0.42      0.36       698
           8       0.11      0.18      0.13       176
           9       0.24      0.21      0.22       227
          10       0.19      0.11      0.14       455
          11       0.24      0.03      0.05      1660
          12       0.35      0.33      0.34      1261
          13       0.49      0.20      0.28       502
          14       0.01      0.04      0.01       156
          15       0.15      0.22      0.18       798
          16       0.46      0.29      0.36   

## **With Mel-Spectrogram, MFCCs, CQT**

In [106]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(42578, 232)

In [107]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(13083, 232)

In [108]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [109]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 0.755413593874771
Validation accuracy: 0.2198272567453948


In [110]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.10      0.19      0.13       793
           1       0.46      0.50      0.48      1294
           2       0.23      0.35      0.27       617
           3       0.15      0.32      0.21       984
           4       0.04      0.04      0.04       229
           5       0.00      0.00      0.00       303
           6       0.11      0.01      0.02       275
           7       0.31      0.42      0.36       698
           8       0.09      0.15      0.12       176
           9       0.23      0.21      0.22       227
          10       0.19      0.11      0.14       455
          11       0.24      0.03      0.05      1660
          12       0.35      0.33      0.34      1261
          13       0.49      0.19      0.28       502
          14       0.01      0.04      0.01       156
          15       0.16      0.23      0.19       798
          16       0.46      0.29      0.36   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [111]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(42578, 160)

In [112]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(13083, 160)

In [113]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [114]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 0.7739677767861337
Validation accuracy: 0.3102499426736987


In [115]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.18      0.45      0.26       793
           1       0.56      0.51      0.53      1294
           2       0.26      0.31      0.29       617
           3       0.28      0.55      0.37       984
           4       0.12      0.05      0.07       229
           5       0.07      0.06      0.07       303
           6       0.14      0.00      0.01       275
           7       0.30      0.36      0.33       698
           8       0.09      0.21      0.13       176
           9       0.35      0.33      0.34       227
          10       0.38      0.44      0.41       455
          11       0.29      0.02      0.04      1660
          12       0.35      0.30      0.32      1261
          13       0.29      0.19      0.23       502
          14       0.05      0.15      0.07       156
          15       0.36      0.40      0.38       798
          16       0.57      0.67      0.62   

# Review results from all models

In [116]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,0.752,0.220
3,chroma,0.358,0.181
8,chroma_cqt,0.889,0.209
4,cqt,0.963,0.198
2,melspectrogram,0.967,0.271
9,melspectrogram_chroma,0.787,0.305
12,melspectrogram_chroma_mfcc,0.774,0.310
10,melspectrogram_cqt,0.762,0.221
11,melspectrogram_cqt_mfcc,0.755,0.220
1,mfcc,0.679,0.273


In [117]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score AUC_Score
0              all_features     0.211      None
3                    chroma     0.160      None
8                chroma_cqt     0.196      None
4                       cqt     0.184      None
2            melspectrogram     0.254      None
9     melspectrogram_chroma     0.287      None
10       melspectrogram_cqt     0.212      None
11  melspectrogram_cqt_mfcc     0.291      None
1                      mfcc     0.257      None
6               mfcc_chroma     0.294      None
7                  mfcc_cqt     0.199      None
5       mfcc_melspectrogram     0.266      None
